# Initial Setup

In [0]:
%sql
drop table if exists genai_usecases.multi_step_tools.sensor_readings_delta

In [0]:
%sql
CREATE TABLE IF NOT EXISTS genai_usecases.multi_step_tools.sensor_readings_delta (
  sensor_id STRING,
  ts TIMESTAMP,
  value DOUBLE
)
USING DELTA;

-- Insert sensor readings across multiple days
INSERT INTO genai_usecases.multi_step_tools.sensor_readings_delta (sensor_id, ts, value) VALUES
-- Day 1 readings
('sensor_A', TIMESTAMP '2025-10-21 08:00:00', 22.3),
('sensor_A', TIMESTAMP '2025-10-21 09:00:00', 23.7),
('sensor_A', TIMESTAMP '2025-10-21 10:00:00', 24.0),
('sensor_A', TIMESTAMP '2025-10-21 11:00:00', 999.9),  -- anomaly spike
('sensor_B', TIMESTAMP '2025-10-21 08:30:00', 49.5),
('sensor_B', TIMESTAMP '2025-10-21 09:30:00', 50.2),

-- Day 2 readings
('sensor_A', TIMESTAMP '2025-10-22 08:00:00', 22.1),
('sensor_A', TIMESTAMP '2025-10-22 09:15:00', 23.5),
('sensor_A', TIMESTAMP '2025-10-22 10:30:00', -5.0),   -- invalid reading
('sensor_B', TIMESTAMP '2025-10-22 08:45:00', 51.0),
('sensor_B', TIMESTAMP '2025-10-22 09:45:00', 49.8),
('sensor_B', TIMESTAMP '2025-10-22 10:45:00', 52.3),

-- Day 3 readings
('sensor_A', TIMESTAMP '2025-10-23 08:10:00', 23.0),
('sensor_A', TIMESTAMP '2025-10-23 09:10:00', 24.5),
('sensor_A', TIMESTAMP '2025-10-23 10:10:00', 25.1),
('sensor_B', TIMESTAMP '2025-10-23 08:40:00', 48.7),
('sensor_B', TIMESTAMP '2025-10-23 09:40:00', 50.1),
('sensor_B', TIMESTAMP '2025-10-23 10:40:00', 5000.0),

-- Today readings
('sensor_A', TIMESTAMP '2025-10-24 18:10:00', 27.0),
('sensor_A', TIMESTAMP '2025-10-24 19:10:00', 24.5),
('sensor_A', TIMESTAMP '2025-10-24 12:10:00', 26.1),
('sensor_B', TIMESTAMP '2025-10-24 14:40:00', 23.7),
('sensor_B', TIMESTAMP '2025-10-24 16:40:00', 98.1),
('sensor_B', TIMESTAMP '2025-10-24 20:40:00', 640.0);  -- crazy outlier


num_affected_rows,num_inserted_rows
24,24


# Install the necessary modules

In [0]:
!pip install langchain databricks-langchain --quiet
dbutils.library.restartPython()

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


# Perform necessary imports

In [0]:
import os
from databricks_langchain import ChatDatabricks
from langchain.tools import tool
from langchain_core.messages import HumanMessage,SystemMessage
from datetime import datetime, timedelta

# Setup the required env variable

In [0]:
os.environ["DATABRICKS_HOST"] = "{Put the Databricks Host Name here}"
os.environ["DATABRICKS_TOKEN"] = dbutils.notebook.entry_point.getDbutils().notebook().getContext().apiToken().getOrElse(None)

# Define the brain of Agent

In [0]:
llm = ChatDatabricks(endpoint="databricks-claude-sonnet-4-5")

# Define the tools

In [0]:
class SensorTools:

    @tool
    def get_max_sensor_value_for_a_day(sensor_id: str, date: str) -> str:
        """
        Retrieves the maximum recorded value for a specific sensor 
        from the Delta table 'sensor_readings_delta'.
        
        Args:
            sensor_id (str): Name/ID of the sensor.
            date (str): Date in 'YYYY-MM-DD' format.
        
        Returns:
            str: The maximum Temperature value as a string.
        """
        df = spark.sql(f"""
            SELECT MAX(value) AS max_value 
            FROM genai_usecases.multi_step_tools.sensor_readings_delta  
            WHERE sensor_id = '{sensor_id}' AND to_date(ts) = to_date('{date}')
        """)
        
        max_val = df.collect()[0]["max_value"]
        return str(max_val) if max_val is not None else "No data found"

    @tool
    def get_current_date() -> str:
        """
        Returns the current timestamp.

        This tool fetches the system's current local date and time. 
        It is useful for tasks where the LLM needs to reference the actual 
        current time instead of reasoning about it contextually.

        Returns:
            str: The current local timestamp in the format YYYY-MM-DD HH:MM:SS.
        """
        return datetime.now().strftime("%Y-%m-%d")


    @tool
    def get_past_date(days_before: int) -> str:
        """
        Returns the date N days before the current date.

        This tool calculates the date that occurred a specific number 
        of days before today. It is handy for temporal comparisons, 
        like fetching sensor readings or logs from a few days back.

        Args:
            days_before (int): Number of days to go back from today.
                            Default is 3.

        Returns:
            str: The date N days before today in the format YYYY-MM-DD.
        """
        past_date = datetime.now() - timedelta(days=days_before)
        return past_date.strftime("%Y-%m-%d")
    

# Initialize the agent

In [0]:
from langchain_classic.agents import AgentType,initialize_agent

agent = initialize_agent(
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    tools=[SensorTools.get_max_sensor_value_for_a_day, SensorTools.get_current_date, SensorTools.get_past_date],
    llm=llm,
    verbose=True,
    max_iterations=5
)

/home/spark-05339830-27b8-4b31-832c-0b/.ipykernel/26284/command-7713863096453299-3745285185:3: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the [LangGraph documentation](https://langchain-ai.github.io/langgraph/) as well as guides for [Migrating from AgentExecutor](https://python.langchain.com/docs/how_to/migrate_agent/) and LangGraph's [Pre-built ReAct agent](https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/).
  agent = initialize_agent(


# Test out

In [0]:
prompt = "What is the max temperature recorded by sensor_B today?"
agent.invoke(prompt)



> Entering new AgentExecutor chain...
I need to find the maximum temperature recorded by sensor_B today. First, I need to get today's date, then query the sensor data.

Action:
```
{
  "action": "get_current_date",
  "action_input": {}
}
```


Observation: 2025-10-27
Thought:Now I have today's date (2025-10-27). I can use this to query the maximum temperature for sensor_B.

Action:
```
{
  "action": "get_max_sensor_value_for_a_day",
  "action_input": {
    "sensor_id": "sensor_B",
    "date": "2025-10-27"
  }
}
```


Observation: No data found
Thought:The query returned no data for sensor_B on today's date (2025-10-27). This could mean that sensor_B hasn't recorded any data today, or there might be no readings available yet.

Action:
```
{
  "action": "Final Answer",
  "action_input": "No temperature data was found for sensor_B today (2025-10-27). This could mean the sensor hasn't recorded any readings yet today, or there may be no data available for this date."
}
```

> Finished cha

{'input': 'What is the max temperature recorded by sensor_B today?',
 'output': "No temperature data was found for sensor_B today (2025-10-27). This could mean the sensor hasn't recorded any readings yet today, or there may be no data available for this date."}

# Check whether it's handling memory or not

In [0]:
prompt = "What is the last question I asked?"
agent.invoke(prompt)



> Entering new AgentExecutor chain...
I don't have access to previous conversation history or memory of past questions. Each interaction starts fresh without context from earlier exchanges.

Action:
```
{
  "action": "Final Answer",
  "action_input": "I don't have access to our previous conversation history. Each interaction I have is independent, so I cannot recall what you asked before this current question. If you'd like to discuss something from earlier, please feel free to repeat or rephrase your question, and I'll be happy to help!"
}
```

> Finished chain.


{'input': 'What is the last question I asked?',
 'output': "I don't have access to our previous conversation history. Each interaction I have is independent, so I cannot recall what you asked before this current question. If you'd like to discuss something from earlier, please feel free to repeat or rephrase your question, and I'll be happy to help!"}

# Import the required module for memory

In [0]:
from langchain_classic.chains.conversation.memory import ConversationBufferMemory

# Define the memory

In [0]:
conversational_memory = ConversationBufferMemory(k=5, return_messages=True)

/home/spark-05339830-27b8-4b31-832c-0b/.ipykernel/26284/command-4531266607502203-1431103100:1: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  conversational_memory = ConversationBufferMemory(k=3, return_messages=True)


# Redefine the agent with memory

In [0]:
from langchain_classic.agents import AgentType,initialize_agent

agent = initialize_agent(
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    tools=[SensorTools.get_max_sensor_value_for_a_day, SensorTools.get_current_date, SensorTools.get_past_date],
    llm=llm,
    verbose=True,
    max_iterations=5,
    memory=conversational_memory
)

# Ask first question to the agent with memory

In [0]:
user_query = input("Enter the User Query: ")
chat_history = conversational_memory.load_memory_variables({}).get('history','')
prompt = f"""
Answer the following questions as best you can.
Chat history: {chat_history}
Question: {user_query}""" 
agent.invoke(prompt)

Enter the User Query:  What is the temperature recorded by sensor_C today?



> Entering new AgentExecutor chain...
I need to find the temperature recorded by sensor_C today. First, I need to get today's date, then query the sensor data for that date.

Action:
```
{
  "action": "get_current_date",
  "action_input": {}
}
```


Observation: 2025-10-27
Thought:Now I have today's date (2025-10-27). I need to get the maximum temperature value recorded by sensor_C for this date.

Action:
```
{
  "action": "get_max_sensor_value_for_a_day",
  "action_input": {
    "sensor_id": "sensor_C",
    "date": "2025-10-27"
  }
}
```


Observation: No data found
Thought:The query returned no data for sensor_C on today's date (2025-10-27). This means either sensor_C hasn't recorded any data today, or there might be no readings available yet for this date.

Action:
```
{
  "action": "Final Answer",
  "action_input": "No temperature data was recorded by sensor_C today (2025-10-27). This could mean the sensor hasn't collected any readings yet today, or there may be no data available

{'input': '\nAnswer the following questions as best you can.\nChat history: []\nQuestion: What is the temperature recorded by sensor_C today?',
 'history': [HumanMessage(content='\nAnswer the following questions as best you can.\nChat history: []\nQuestion: What is the temperature recorded by sensor_C today?', additional_kwargs={}, response_metadata={}),
  AIMessage(content="No temperature data was recorded by sensor_C today (2025-10-27). This could mean the sensor hasn't collected any readings yet today, or there may be no data available for this date.", additional_kwargs={}, response_metadata={})],
 'output': "No temperature data was recorded by sensor_C today (2025-10-27). This could mean the sensor hasn't collected any readings yet today, or there may be no data available for this date."}

# Ask second question to the agent which is related to memory

In [0]:
user_query = input("Enter the User Query: ")
chat_history = conversational_memory.load_memory_variables({}).get('history','')
prompt = f"""
Answer the following questions as best you can.
Chat history: {chat_history}
Question: {user_query}""" 
agent.invoke(prompt)

Enter the User Query:  What is the last question I asked?



> Entering new AgentExecutor chain...
I need to look at the chat history to identify the last question the human asked.

Looking at the chat history provided, I can see:
- The human's last question was: "What is the temperature recorded by sensor_C today?"
- My response indicated that no temperature data was recorded by sensor_C on 2025-10-27.

Now the human is asking what their last question was.

Action:
```
{
  "action": "Final Answer",
  "action_input": "Your last question was: 'What is the temperature recorded by sensor_C today?'"
}
```

> Finished chain.


{'input': '\nAnswer the following questions as best you can.\nChat history: [HumanMessage(content=\'\\nAnswer the following questions as best you can.\\nChat history: []\\nQuestion: What is the temperature recorded by sensor_C today?\', additional_kwargs={}, response_metadata={}), AIMessage(content="No temperature data was recorded by sensor_C today (2025-10-27). This could mean the sensor hasn\'t collected any readings yet today, or there may be no data available for this date.", additional_kwargs={}, response_metadata={})]\nQuestion: What is the last question I asked?',
 'history': [HumanMessage(content='\nAnswer the following questions as best you can.\nChat history: []\nQuestion: What is the temperature recorded by sensor_C today?', additional_kwargs={}, response_metadata={}),
  AIMessage(content="No temperature data was recorded by sensor_C today (2025-10-27). This could mean the sensor hasn't collected any readings yet today, or there may be no data available for this date.", add